This notebook contains calculations for the list of genus 6 hyperelliptic curves over $\mathbf{F}_2$ and among them, which are supersingular curves. The approach follows X. Xarles's "A census of all genus 4 curves over the field with 2 elements".

$Q_6(F_2)$ list

In [1]:
F = GF(2)
R1.<x> = PolynomialRing(F)
Q6list = [
          1,x,
          x^2, x*(x+1),x^2 + x + 1,
          x^3,(x^2)*(x+1),x*(x^2 + x + 1), x^3 + x + 1,
          (x^3)*(x+1), (x^2)*(x+1)^2, (x^2) *(x^2 + x + 1), (x^2 + x + 1)*x*(x+1),(x^2 + x + 1)^2 ,
          (x^3 + x + 1)*x, (x^3 + x^2 + 1)*x, x^4 + x + 1, x^4 + x^3 + 1,
          (x^2)*(x+1)*(x^2 + x + 1),((x^2 + x + 1)^2)*x,(x^3 + x + 1)*x^2, (x^3 + x^2 + 1)*x^2,
          (x^3 + x + 1)*x*(x+1), (x^3 + x + 1)*(x^2 + x + 1), (x^4 + x + 1)*x,(x^4 + x^3 + 1)*x,
          (x^4 + x^3 + x^2 + x + 1)*x,x^5 + x^2 + 1,x^5 + x^3 + 1,x^5 + x^3 + x^2 + x + 1,
          ((x^2 + x + 1)^2)*x*(x+1),(x^2 + x + 1)^3,(x^3 + x + 1)*(x^2 + x + 1)*x,
          (x^3 + x^2 + 1)*(x^2 + x + 1)*x,(x^3 + x + 1)^2,(x^3 + x + 1)*(x^3 + x^2 + 1),
          (x^4 + x + 1)*x*(x+1), (x^4 + x + 1)*(x^2 + x + 1),(x^4 + x^3 +1)*(x^2 + x + 1),
          (x^5 + x^2 + 1)*x,(x^5 + x^4 + x^2 + x + 1)*x, (x^5 + x^4 + x^3 + x^2 + 1)*x,
          (x^6 + x + 1), (x^6 + x^3 + 1), (x^6 + x^5 + 1), (x^6 + x^5 + x^3 + x^2 + 1),
          (x^6 + x^5 + x^4 + x + 1),(x^3 + x + 1)*((x^2 + x + 1)^2),(x^4 + x + 1)*(x^3 + x + 1),
          (x^5 + x^2 + 1)*(x^2 + x + 1),(x^7 + x + 1), (x^7 + x^3 + 1), 
          (x^7 + x^4 + x^3 + x^2 + 1)
         ]

Action of $GL_2(\mathbf{F}_2)$ on $\mathbf{F}_2[x]$

In [2]:
G = GL(2,F)
Glst = list(G.list())
def gaction(level,g,pol):
    g = g.matrix()
    return (g[1][0]*x + g[1][1])^(level)*pol((g[0][0]*x + g[0][1])/(g[1][0]*x + g[1][1]))

Algorithm of Xarles at the end of section 1

In [3]:
D14 = set(R1.polynomials(max_degree=14))

In [4]:
%%time

from collections import defaultdict

Vq = defaultdict(list)
for q in Q6list:
    
    R1q = R1(q)
    
    #compute stabiliser G_q of q
    Gq = []
    if q == 1:
        Gq = [Glst[0],Glst[3]] # 1 is fixed under [1,0,0,1] and [1,1,0,1]
    else:
        for g in Glst:
            if gaction(7,g,R1q) == R1q:
                Gq.append(g)
        
    #find representative p of isomorphism class [y^2 + qy = p]
    D14copy = D14.copy()
    while D14copy:
        p = D14copy.pop()
        m = max(2*R1q.degree(),p.degree())
        if 13 <= m and m <= 14:
            Spprime = set([p + r^2 + R1q*r for r in R1.polynomials(max_degree=7)])

            Sp = set()
            for pol in Spprime:
                Sp.update([gaction(14,g,pol) for g in Gq])
            D14copy = D14copy - Sp
            ele = Sp.pop()
            try:
                Vq[R1q].append(ele)
            except KeyError:
                Vq[R1q] = [ele]            
        else:
            continue

CPU times: user 25min 14s, sys: 474 ms, total: 25min 15s
Wall time: 25min 18s


Check that the equation $y^2 + q(x)y = p(x)$ defines a nonsingular hyperelliptic curve:

For
- 0 $\not = q $ with $1 \leq deg(q) \leq g + 1 = 7$
- $p$ with $deg(p) \leq 2g + 2$
- $13 = 2g + 1 \leq max(2deg(q),deg(p)) \leq 2g + 2 = 14$

Then the above equation is nonsingular iff
- $gcd(q,p'^2 + q'^2p) = 1$
- either $deg(q) = g+1$ or $a_{2g+1}^2 \not = a_{2g+2}b_g^2$ where $p = \sum_{i=0}^{2g+2} a_i x^i, q = \sum_{i=0}^{g+1} b_ix^i$

In [5]:
%%time

R.<x> = PolynomialRing(QQ)
qphypell = {}

for q in Vq:
    plst = []
    for p in Vq[q]:
        if gcd(q, p.derivative()^2 + (q.derivative()^2)*p) == 1 and (R(q).degree() == 7 or mod(R(p).monomial_coefficient(x^13),2)^2 != mod((R(p).monomial_coefficient(x^14)),2)*mod(R(q).monomial_coefficient(x^6),2)^2):
            plst.append(p)

    qphypell[q] = plst

CPU times: user 2.65 s, sys: 4 ms, total: 2.65 s
Wall time: 2.65 s


In [6]:
sum(len(qphypell[key]) for key in qphypell)

4232

In [7]:
f = open("data/unfiltered.txt","w+")
for q in qphypell:
    for p in qphypell[q]:
        to_write = "y^2 + (" + str(q) + ")*y + " + str(p) + "\n"
        f.write(to_write)
f.close()